In [114]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e4/sample_submission.csv
/kaggle/input/playground-series-s5e4/train.csv
/kaggle/input/playground-series-s5e4/test.csv


In [115]:
train = pd.read_csv('/kaggle/input/playground-series-s5e4/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s5e4/test.csv')

In [116]:
categorical = ['Podcast_Name', 'Episode_Title', 'Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']
numerical = ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Number_of_Ads', 'Listening_Time_minutes']

categorical_test = ['Podcast_Name', 'Episode_Title', 'Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']
numerical_test = ['Episode_Length_minutes', 'Host_Popularity_percentage', 'Guest_Popularity_percentage', 'Number_of_Ads']

In [117]:
train.isnull().sum()

id                                  0
Podcast_Name                        0
Episode_Title                       0
Episode_Length_minutes          87093
Genre                               0
Host_Popularity_percentage          0
Publication_Day                     0
Publication_Time                    0
Guest_Popularity_percentage    146030
Number_of_Ads                       1
Episode_Sentiment                   0
Listening_Time_minutes              0
dtype: int64

In [118]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

def feature_eng(df):
    return df

def preprocess(df, categorical, numerical):
    df = df.copy()

    num_imputer = SimpleImputer(strategy = 'median')
    df[numerical] = num_imputer.fit_transform(df[numerical])
    df[categorical] = df[categorical].fillna('Missing')
    
    df = pd.get_dummies(df, columns=categorical, drop_first=True)

    return df

In [119]:
train = preprocess(train, categorical, numerical)

In [120]:
x_train = train.drop(columns = ['id', 'Listening_Time_minutes'])
y_train = train['Listening_Time_minutes']

In [121]:
import xgboost as xgb

xgb = xgb.XGBRegressor(
    n_estimators=100,       # how many trees to build
    learning_rate=0.1,      # how fast we learn; lower = safer
    max_depth=6,            # how deep each tree goes
    subsample=0.8,          # % of data to sample per tree (boosts generalization)
    colsample_bytree=0.8,   # % of features per tree (prevents overfitting)
    random_state=42,        # so results don’t change every time
    n_jobs=-1,              # use all the cores like a multitasking queen
    verbosity=2             # 0 = silent, 1 = warning, 2 = info, 3 = debug
)

xgb.fit(x_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=-1,
             num_parallel_tree=None, random_state=42, ...)

In [122]:
test = preprocess(test, categorical_test, numerical_test)

In [123]:
x_test = test.drop(columns = ['id'])

In [124]:
preds = xgb.predict(x_test)
test['preds'] = preds

In [125]:
submission = pd.DataFrame({
    'id' : test['id'],
    'Listening_Time_minutes' : test['preds']
})

In [126]:
submission.to_csv('submission.csv', index = False)